In [71]:
import tensorflow as tf
from tensorflow.keras import models, layers
import matplotlib.pyplot as plt

import numpy as np

In [2]:
IMAGE_SIZE = 256
BATCH_SIZE = 32
CHANNELS = 3
EPOCHS = 10

In [ ]:
dataset = tf.keras.preprocessing.image_dataset_from_directory(
    "PlantVillage",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE    
)

In [ ]:
class_names = dataset.class_names
class_names

In [ ]:
len(dataset)

In [ ]:
for image_batch,label_batch in dataset.take(1):
    print(image_batch.shape)
    print(label_batch.numpy())
    print(image_batch[0].shape)
    # print(image_batch[0].numpy())
    plt.imshow(image_batch[0].numpy().astype("uint8"))
    plt.title(class_names[label_batch[0]])

In [ ]:
plt.figure(figsize=(10,10))
for image_batch,label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3,4,i+1)
        plt.imshow(image_batch[i].numpy().astype('uint8'))
        plt.title(class_names[label_batch[i]])
        plt.axis("off")

* TRAINING - 80%
* VALIDATION - 10%
* TEST - 10%

In [ ]:
train_size = 0.8
val_size = 0.1
test_size = 0.1


int(len(dataset)*train_size)

In [ ]:
train_ds = dataset.take(int(len(dataset)*train_size))
test_val = dataset.skip(int(len(dataset)*train_size))
val_ds = test_val.take(int(len(dataset)*val_size))
test_ds = test_val.skip(int(len(dataset)*val_size))

print(f"train - {len(train_ds)} samples")
print(f"validation - {len(val_ds)} samples")
print(f"test - {len(test_ds)} samples")

In [10]:
def get_dataset_partitions_tf(ds, train_split = 0.8, val_split = 0.1, test_size = 0.1, shuffle=True, shuffle_size=10000):
    if shuffle:
        ds = ds.shuffle(shuffle_size,seed=18)
        
    train_ds = ds.take(int(len(ds)*train_split))
    test_val = ds.skip(int(len(ds)*train_split))
    val_ds = test_val.take(int(len(ds)*val_split))
    test_ds = test_val.skip(int(len(ds)*val_split))
    
    return train_ds, val_ds, test_ds
    

In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)
print(f"train - {len(train_ds)} samples")
print(f"validation - {len(val_ds)} samples")
print(f"test - {len(test_ds)} samples")

In [ ]:
#Caching - loading the image in memory
#prefetch - while GPU is training, CPU will pre load some images
train_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
val_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)
test_ds.cache().shuffle(1000).prefetch(buffer_size = tf.data.AUTOTUNE)

In [13]:
#Normalize the data - Layers - devide by 255
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE,IMAGE_SIZE),
    layers.Rescaling(1.0/255)
])

In [14]:
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2)
])

## Model Building

In [ ]:
# (32, 256, 256, 3)
(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)

In [42]:
input_shape = (BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes = 3

model = models.Sequential([
    layers.Input(shape=(IMAGE_SIZE, IMAGE_SIZE, CHANNELS),batch_size=BATCH_SIZE),
    resize_and_rescale,
    data_augmentation,
    layers.Conv2D(32,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])

model.build(input_shape=input_shape)

In [ ]:
model.summary()

In [44]:
model.compile(
    optimizer='adam',
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    epochs = EPOCHS,
    batch_size = BATCH_SIZE,
    verbose = 1,
    validation_data = val_ds
)

In [ ]:
model.evaluate(test_ds)

In [ ]:
history.history.keys()

In [55]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(10,3))
plt.subplot(1,2,1)
plt.plot(range(EPOCHS),acc,label = 'training_acc')
plt.plot(range(EPOCHS),val_acc,label = 'val_acc')
plt.legend(loc='lower right')
plt.title("training and validation accuracy")

plt.subplot(1,2,2)
plt.plot(range(EPOCHS),loss,label = 'training_loss')
plt.plot(range(EPOCHS),val_loss,label = 'val_loss')
plt.legend(loc='lower right')
plt.title("training and validation accuracy")

In [ ]:
for images_batch,label_batch in test_ds.take(1):
    first_img = image_batch[0].numpy().astype('uint8')
    first_label = label_batch[0]
    
    print("first image to predict")
    plt.imshow(first_img)
    print(f"first image's actual label: {class_names[first_label]}")
    
    batch_prediction = model.predict(images_batch)
    print(f"first image's predicted label: {class_names[np.argmax(batch_prediction[0])]}")

In [92]:
def predict(model, img):
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0)
    
    predictions = model.predict(img_array)
    
    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.argmax(predictions[0])), 2)
    return predicted_class,confidence

In [ ]:
plt.figure(figsize=(10,10))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i+1)
        plt.imshow(images[i].numpy().astype('uint8'))
        
        predicted_class,confidence = predict(model, images[1].numpy())
        actual_class = class_names[labels[i]]
        plt.title(f"{predicted_class},{confidence},\n{actual_class}")
        plt.axis("off")

In [ ]:
import os

model_version = 1
model_folder = r"C:\Users\nigam\OneDrive\Documents\self\Potato-Disease-Classification-using-CNN\models"
model_path = os.path.join(model_folder, f"{model_version}")  # Use .keras extension

model.export(model_path)